### This notebook shows how to create optional files for an event.

Import required packages

In [14]:
import pandas as pd
import numpy as np
from astropy.time import Time
import re
import os

##### Creating fields.txt

In [9]:
data_20230627 = pd.read_csv('event_20230627/2023-06-27_v5.csv', sep=',') # Load the event data file.

Using information on gracedb, set up the $t_{start}$ and $t_{end}$ for the analysis. Use $t_0$ or coalescence time as $t_{start}$ set $t_{end}$ to be three days after $t_{start}$.

In [10]:
t_start = '2023-06-27T01:53:37.819'
t_end = '2023-06-30T01:53:37.819'

# Convert t_start and t_end to Time in isot format
t_start = Time(t_start,format='isot')
t_end = Time(t_end,format='isot')

Since we are only concerned with observations between $t_{start}$ and $t_{end}$, create a data_event file using the start and end time in jd format.

In [11]:
data_event = data_20230627[(data_20230627['jd']>=t_start.jd)&(data_20230627['jd']<=t_end.jd)]

Find all the unique fields in the observation file with status $1$. nimbus only takes those observations into account that have a status of $1$. status $1$ indicates a complete observation.

In [13]:
fields = np.unique(data_event[data_event['status']==1]['field'])
fields = fields.astype('int')

Save the fields in a file. This will be useful while running the jobs as the jobs can be queued for all the fields in an event.

In [ ]:
file = open('fields.txt', 'w+')
for field in fields:
    content = str(field)
    file.write(content)
    file.write('\n')
file.close()

The next part of this notebook deals with files that can be generated once the likelihood files are obtained.

Set up the directory.

In [15]:
directory = 'event_20230518/likelihood/'

Some likelihood files can be from fields with $E(B-V) >2$ and nimbus returns an error in that case in the .err files. To keep a note of those fields, create a status.txt file.

In [23]:
# Since there is a lot of file reading to do:
def read_text_file(file_path): 
    with open(file_path, 'r') as f: 
        content = f.read()
    return content

# Read a file that has too much extinction (added in the repository)
extinction = read_text_file('/home/vaidehi.gupta/nimbus_analysis/O4_nimbus/event_20230521/err/multi_field_1636.err')

# Store the name of any .err file which has the same content.
extinction_files = []
for file in os.listdir(directory):
    file_path = f"{directory}{file}"
    content = read_text_file(file_path)
    if content == extinction:
        extinction_files.append(file)

For field number $880$ and $881$, the ZTF_fields.pkl file, doesn't have any information. If these fields appear in the observation file, store that in the status file as well.

In [ ]:
with open('status.txt', 'w+') as f:
    for fld in fields:
        if fld == 880 or fld == 881:
            content = f"{fld}: No extinction information found"
            f.write(content)
            f.write('\n')
    for file in extinction_files:
        field = [int(s) for s in re.findall(r'\d+', file)]
        for fld in field:
            content = f"{fld}: extinction"
            f.write(content)
            f.write('\n')     
    f.close()

Cross-check if you have been successfully able to generate all the meaningful likelihood files.

In [ ]:
directory = '/home/vaidehi.gupta/nimbus_analysis/O4_nimbus/event_20230627/likelihood/'
fields_done = []
for file in os.listdir(directory):
    fields_done.append([int(s) for s in re.findall(r'\d+', file)][0])

print(len(fields), len(fields_done))

for f in fields:
    if f not in fields_done:
        print(f)